# Pytorch Tutorial

Pytorch is a popular deep learning framework and it's easy to get started.

In [134]:
import torch
import torch.nn as nn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
import time

BATCH_SIZE = 128
NUM_EPOCHS = 10

First, we read the mnist data, preprocess them and encapsulate them into dataloader form.

In [135]:
# preprocessing
normalize = transforms.Normalize(mean=[.5], std=[.5])
transform = transforms.Compose([transforms.ToTensor(), normalize])

# download and load the data
train_dataset = torchvision.datasets.MNIST(root='./mnist/', train=True, transform=transform, download=True)
test_dataset = torchvision.datasets.MNIST(root='./mnist/', train=False, transform=transform, download=False)

# encapsulate them into dataloader form
train_loader = data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
test_loader = data.DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, drop_last=True)

Then, we define the model, object function and optimizer that we use to classify.

In [136]:
class SimpleNet(nn.Module):
# TODO:define model
    def __init__(self):
        super(SimpleNet, self).__init__()
        self.conv1 = nn.Sequential(
            nn.Conv2d(1, 6, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(6, 16, 5),
            nn.ReLU(),
            nn.MaxPool2d(2, 2)
        )
        self.fc1 = nn.Sequential(
            nn.Linear(16 * 4 * 4, 128),
            nn.ReLU(),
            nn.Dropout2d(p=0.3)
        )
        #self.fc2 = nn.Sequential(
        #    nn.Linear(120, 84),
        #    nn.ReLU(),
        #    nn.Dropout2d(p=0.3)
        #)
        self.fc3 = nn.Sequential(
            nn.Linear(128, 10)
        )
    
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        #x = self.fc2(x)
        x = self.fc3(x)
        return x

model = SimpleNet()

# TODO:define loss function and optimiter
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=1e-3)
#optimizer = torch.optim.SGD(model.parameters(),lr=0.1)
#optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

Next, we can start to train and evaluate!

In [139]:
# train and evaluate
for epoch in range(NUM_EPOCHS):
    for images, labels in tqdm(train_loader):
        # TODO:forward + backward + optimize
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
    # evaluate
    # TODO:calculate the accuracy using traning and testing dataset
    with torch.no_grad():
        true_preds = 0
        train_loss = 0
        for images, labels in tqdm(train_loader):
            outputs = model(images)
            train_preds = torch.max(outputs.data, 1)[1]
            train_loss += criterion(outputs, labels)
            true_preds += (train_preds == labels).sum()
        print('Training loss: %.4f, Training accuracy: %.2f%%' % (train_loss, 100 * true_preds / len(train_loader.dataset)))
        true_preds = 0
        test_loss = 0
        for images, labels in tqdm(test_loader):
            outputs = model(images)
            test_preds = torch.max(outputs.data, 1)[1]
            test_loss += criterion(outputs, labels)
            true_preds += (test_preds == labels).sum()
        print('Testing loss: %.4f, Testing accuracy: %.2f%%' % (test_loss, 100 * true_preds / len(test_loader.dataset)))

100%|██████████| 78/78 [00:01<00:00, 58.75it/s]Testing loss: 2.8074, Testing accuracy: 98.73%



#### Q5:
Please print the training and testing accuracy.

In [142]:
true_preds = 0
train_loss = 0
for images, labels in tqdm(train_loader):
    outputs = model(images)
    train_preds = torch.max(outputs.data, 1)[1]
    train_loss += criterion(outputs, labels)
    true_preds += (train_preds == labels).sum()
print('\nTraining loss: %.4f, Training accuracy: %.2f%%' % (train_loss, 100 * true_preds / len(train_loader.dataset)))

true_preds = 0
test_loss = 0
for images, labels in tqdm(test_loader):
    outputs = model(images)
    test_preds = torch.max(outputs.data, 1)[1]
    test_loss += criterion(outputs, labels)
    true_preds += (test_preds == labels).sum()
print('\nTesting loss: %.4f, Testing accuracy: %.2f%%' % (test_loss, 100 * true_preds / len(test_loader.dataset)))

  8%|▊         | 6/78 [00:00<00:01, 54.37it/s]
Training loss: 7.8349, Training accuracy: 99.30%
100%|██████████| 78/78 [00:01<00:00, 53.80it/s]
Testing loss: 3.0242, Testing accuracy: 98.65%

